In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt
# import P
import os
# import openCV as 

In [2]:
data_dir = 'E:\ELL_Project\problem3\\'
lis_folders = os.listdir(data_dir+'.')[:-1]
X = []
cur_class = 0
for folder in lis_folders:
    i=0
    lis_files = os.listdir(data_dir+folder+'\.')
    im = []
    for files in lis_files:
        i+=1
        if(i==1001):
            break
        img = '\\'.join([data_dir,folder,files])
        image = plt.imread(img)
        im.append(image.flatten().reshape(-1,1))
    im_np = np.array(im)[:,:,0]
    # print(im_np.shape)
    im_np = np.append(im_np,cur_class*np.ones((im_np.shape[0],1)),axis=1)
    # print(im_np.shape)
    cur_class+=1
    X.append(im_np)


In [3]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=2)
X_train = X[0]
for x in X[1:]: 
    X_train = np.append(X_train,x,axis=0)
X_pca = X_train
pca.fit(X_pca[:,:-1])
X_pca[:,-3:-1] = pca.transform(X_pca[:,:-1])
print(X_pca.shape)

(6000, 4097)


In [4]:
X_pcaa = X_pca[:,-3:]
np.random.shuffle(X_pcaa)

def import_dataset(X_pcaa,split_percent = 70):
    
    X = X_pca[:,:-1]
    y = X_pca[:,-1:]
    datasize = X.shape[0]

    split_point = split_percent//10

    X_train = X[:(datasize*split_point)//10,:]
    y_train = y[:(datasize*split_point)//10,:]
    X_test = X[(datasize*split_point)//10:,:]
    y_test = y[(datasize*split_point)//10:,:]

    return(X_train,X_test,y_train,y_test)

In [5]:
print(X_pca[:30,:])

[[ 1.01000000e+02  1.01000000e+02  1.01000000e+02 ... -3.62889203e+03
  -6.87125431e+02  5.00000000e+00]
 [ 1.01000000e+02  1.01000000e+02  1.01000000e+02 ... -2.86718464e+03
  -3.24344175e+02  5.00000000e+00]
 [ 1.01000000e+02  1.01000000e+02  1.01000000e+02 ...  2.51602163e+03
  -1.30904317e+03  2.00000000e+00]
 ...
 [ 1.01000000e+02  1.01000000e+02  1.01000000e+02 ... -4.03586547e+03
  -8.08821199e+02  1.00000000e+00]
 [ 1.01000000e+02  1.01000000e+02  1.01000000e+02 ...  1.32648620e+03
  -1.53267198e+03  0.00000000e+00]
 [ 1.01000000e+02  1.01000000e+02  1.01000000e+02 ... -2.64290005e+03
   2.44583509e+02  5.00000000e+00]]


In [6]:
# X_dash = X
# # X_dash[1] = X_updated.reshape(-1,4096,1)
# X_np = np.array(X_dash)
# print(X_np.shape)
# X_train = X[0]
# for x in X[1:]:
#     X_train = np.append(X_train,x,axis=0)
# print(X_train.shape)

In [7]:
# X_edit = X[1]
# X_edit = np.array(X_edit)
# X_mean = np.sum(X_edit, axis=0) / X_edit.shape[0]
# X_mean = X_mean.reshape(1,-1)
# print(X_mean.shape)
# diff = X[0].shape[0]-X[1].shape[0]
# oneer = np.ones((diff,1))
# X_app = np.dot(oneer,X_mean)
# X_updated = np.append(X_edit,X_app,axis=0)
# print(X_updated.shape)

In [8]:
def feature_scaling(X_train):
    training_size = X_train.shape[0]
    X_mean = np.sum(X_train,axis=0) / training_size
    X_var = np.sqrt(np.sum((np.square(X_train-X_mean)),axis=0)/training_size)
    X_train_reg = (X_train - X_mean) / X_var

    return (X_mean,X_var,X_train_reg)


In [9]:
def KNN(X_train,y_train,X_test,y_test,k_num,feat_scal=False):

    num_class = 6
    test_size = X_test.shape[0]
    train_size = X_train.shape[0]
    if feat_scal:
        X_m,X_v,X_r = feature_scaling(X_train)
        X_test_reg = (X_test-X_m)/X_v
    else:
        X_r = X_train
        X_test_reg = X_test
    
    y_pred = []
    
    for i in range(test_size):
        sample_X = X_test_reg[i:i+1,:]
        sample_y = y_test[i,:]
        dist_vec = np.sqrt(np.sum(np.square(X_r-sample_X),axis=1))
        dist_vec = dist_vec.reshape((train_size,1))

        d = np.column_stack((dist_vec[:,:],y_train[:,0]))
        # print(d)

        sorted_dist = d[np.argsort(d[:, 0])]
        y_val_vector = sorted_dist[:k_num,1].reshape(-1,1)
        class_sizes = []
        for j in range(num_class):
            num = y_val_vector[ y_val_vector[:,0]==j ].shape[0]
            class_sizes.append([num,j])
        class_sizes.sort(reverse=True)
        y_pred.append(class_sizes[0][1])   
    
    y_pred = np.array(y_pred).reshape(-1,1)

    # print(y_pred.shape)
    return y_pred
    
        



In [10]:
def accuracy(y_pred,y_test):

    # y_pred = y_pred.reshape(-1,1)
    print('MSE LOSS')
    mse_loss = np.sum(np.square((y_test-y_pred)),axis=0) / y_test.shape[0]
    print(mse_loss)
    print('............................................................................')
    print('MAE LOSS')
    mae_loss = np.sum(np.abs((y_test-y_pred)),axis=0) / y_test.shape[0]
    print(mae_loss)
    # print('............................................................................')

In [17]:
def accuracy(y_pred,y_test):

    test_size = y_test.shape[0]

    y_pred = y_pred.reshape((y_pred.shape[0],1))
    acc = y_test[ y_pred[:,0] == y_test[:,0] ].shape[0] * 100 / test_size 
    print('Accuracy: {}'.format( acc  ))

In [11]:
X_train,X_test,y_train,y_test = import_dataset(X_pcaa)
print('model run initiated...')
y_pred = KNN(X_train,y_train,X_test,y_test,50,False)
print('model run completed')
accuracy(y_pred,y_test)

model run initiated...
model run completed
MSE LOSS
[0.44888889]
............................................................................
MAE LOSS
[0.17888889]


In [18]:
accuracy(y_pred,y_test)

Accuracy: 91.38888888888889


In [12]:
print(y_test.shape)

(1800, 1)
